In [1]:
from gensim.models import Doc2Vec, Word2Vec
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
pd.set_option('display.max_colwidth', -1)

In [2]:
dirpath = "C:/Users/D072202/DeepAnyMatch/DeepAnyMatch/result_data/OAEI_2019_06_10_16_53_26_358244/"
#model = Doc2Vec.load(dirpath+"w2v.model")
model = Word2Vec.load(dirpath+"w2v.model")
documents_ids_A = dict()
documents_ids_B = dict()
training_corpus = dict()
#with open(dirpath+"document_ids.csv") as f:
#    for line in f:
#        if '/hilti_erp/' in line.split(",")[0]:
#            documents_ids_A[int(line.split(",")[1])] = str(line.split(",")[0])
#        else:
#            documents_ids_B[int(line.split(",")[1])] = str(line.split(",")[0])
#ctr = 0
#with open(dirpath+"w2v_training_material.csv", encoding="UTF-8") as f:
#    for line in f:
#        training_corpus[ctr] = line.split(' ')
#        ctr += 1

possible_matches_original = dict()
with open('C:/Users/D072202/DeepAnyMatch/DeepAnyMatch/data/oaei_data/possible_matches.csv', encoding="UTF-8") as f:
    for line in f:
        line = line.replace("\n","").split("\t")
        if line[0] in possible_matches_original.keys():
            possible_matches_original[line[0]].add(line[1])
        else:
            possible_matches_original[line[0]] = set([line[1]])
            
        if line[1] in possible_matches_original.keys():
            possible_matches_original[line[1]].add(line[0])
        else:
            possible_matches_original[line[1]] = set([line[0]])

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/D072202/DeepAnyMatch/DeepAnyMatch/result_data/OAEI_2019_06_10_16_53_26_358244/w2v.model'

In [662]:
resourceclasses = pd.read_csv(dirpath+'stratified_embeddings.csv')
resources = resourceclasses.loc[~(resourceclasses.category=='http://www.w3.org/1999/02/22-rdf-syntax-ns#property') & ~(resourceclasses.category=='http://www.w3.org/2002/07/owl#class')]
classes = resourceclasses.loc[(resourceclasses.category=='http://www.w3.org/2002/07/owl#class')]
properties = resourceclasses.loc[(resourceclasses.category=='http://www.w3.org/1999/02/22-rdf-syntax-ns#property')]
resources.head()

,Unnamed: 0,x0,x1,x2,x3,x4,x5,x6,x7,x8,...,x13,x14,x15,x16,x17,x18,x19,label,category,origin
0,0,-1.451712,-0.473165,-1.213369,-0.272864,-0.305540,-0.235766,-0.968213,1.014426,-0.351985,...,-1.328647,-1.269559,1.451907,-0.178004,-0.782943,0.955942,1.049185,http://darkscape.wikia.com/wiki/special:filepath/cait.png?width=300,http://dbkwik.webdatacommons.org/ontology/image,graph1
1,1,-1.451712,-0.473165,-1.213369,-0.272864,-0.305540,-0.235766,-0.968213,1.014426,-0.351985,...,-1.328647,-1.269559,1.451907,-0.178004,-0.782943,0.955942,1.049185,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,none,graph1
2,2,-1.451712,-0.473165,-1.213369,-0.272864,-0.305540,-0.235766,-0.968213,1.014426,-0.351985,...,-1.328647,-1.269559,1.451907,-0.178004,-0.782943,0.955942,1.049185,http://dbkwik.webdatacommons.org/ontology/image,none,graph1
3,3,-0.788001,0.572844,-0.271577,0.740157,-0.424576,-1.079205,0.219246,1.446069,0.479661,...,0.586782,-0.595855,0.984143,0.637527,-0.410277,-0.354513,0.420836,http://dbkwik.webdatacommons.org/darkscape/resource/crystal_tree,http://dbkwik.webdatacommons.org/darkscape/class/tree,graph1
4,4,-1.451712,-0.473165,-1.213369,-0.272864,-0.305540,-0.235766,-0.968213,1.014426,-0.351985,...,-1.328647,-1.269559,1.451907,-0.178004,-0.782943,0.955942,1.049185,http://xmlns.com/foaf/0.1/depiction,none,graph1


## Instance Matching

In [663]:
def get_possible_matches(nid):
        #if DOCVEC:
        #    out = list()
        #    for index, key in documents_ids_B.items():
        #        if key in possible_matches[nid]:
        #            out.append(index)
        #    return out
        #else:
            final_matches = list()
            matches = list(possible_matches_original[nid])
            if nid in resources.label.tolist():
                for m in matches:
                    if m in resources.label.tolist():
                        final_matches.append(m)

            if nid in classes.label.tolist():
                for m in matches:
                    if m in classes.label.tolist():
                        final_matches.append(m)
            
            if nid in properties.label.tolist():
                for m in matches:
                    if m in properties.label.tolist():
                        final_matches.append(m)
                        
            return final_matches

In [664]:
matchings = None

In [688]:
nodeid = 'http://dbkwik.webdatacommons.org/darkscape/property/maxranged'
#'http://dbkwik.webdatacommons.org/darkscape/resource/falador_shield_3'
#'http://dbkwik.webdatacommons.org/darkscape/resource/Guthix_full_helm'
#'http://dbkwik.webdatacommons.org/darkscape/resource/Adamant_shield_(h3)'
#'http://dbkwik.webdatacommons.org/darkscape/resource/2014'
#'http://dbkwik.webdatacommons.org/darkscape/resource/Short_vine'
#'http://dbkwik.webdatacommons.org/darkscape/resource/Prayer'
#'http://dbkwik.webdatacommons.org/darkscape/resource/Ancient_platelegs'

#'http://dbkwik.webdatacommons.org/darkscape/resource/Mithril_2h_sword'
#'http://dbkwik.webdatacommons.org/darkscape/class/Conquest'
#'http://dbkwik.webdatacommons.org/darkscape/class/Recipe'#"http://dbkwik.webdatacommons.org/darkscape/resource/Seers'_Village"
#'http://dbkwik.webdatacommons.org/darkscape/resource/Monkey_talisman'#http://dbkwik.webdatacommons.org/darkscape/resource/Lumbridge_and_Draynor_Tasks'#'http://dbkwik.webdatacommons.org/darkscape/resource/void_knight_equipment'
nodeid = nodeid.lower()
#for index, value in documents_ids_A.items():
#    if value == nodeid:
#        i=index
#print("Targeted "+documents_ids_A[i] + " at i="+str(i))

In [689]:
print(nodeid)
model.wv.most_similar(nodeid)

http://dbkwik.webdatacommons.org/darkscape/property/maxranged


[('http://dbkwik.webdatacommons.org/darkscape/property/affranged',
  0.9840847849845886),
 ('http://dbkwik.webdatacommons.org/darkscape/property/affmelee',
  0.9749705195426941),
 ('http://dbkwik.webdatacommons.org/darkscape/property/affweakness',
  0.9729663729667664),
 ('http://dbkwik.webdatacommons.org/darkscape/property/affmagic',
  0.9687404632568359),
 ('http://dbkwik.webdatacommons.org/darkscape/property/maxmelee',
  0.9680567979812622),
 ('http://dbkwik.webdatacommons.org/darkscape/property/maxmagic',
  0.9672887921333313),
 ('http://dbkwik.webdatacommons.org/darkscape/property/accranged',
  0.9545732140541077),
 ('http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/earth_warrior_champion',
  0.9329987168312073),
 ('232^^http://www.w3.org/2001/xmlschema#integer116^^http://www.w3.org/2001/xmlschema#integer',
  0.9257487654685974),
 ('http://dbkwik.webdatacommons.org/darkscape/property/accmelee',
  0.9254863262176514)]

In [690]:
#get_possible_matches(nodeid)
from sklearn.linear_model import LogisticRegression
possible_matches = pd.read_csv(dirpath + "possible_matches.csv-strcombined.csv", sep=",", encoding="UTF-8")
possible_matches_ids = pd.read_csv(dirpath + "possible_matches.csv-strcombined_ids.csv", sep=",", encoding="UTF-8")
possible_matches = possible_matches.merge(possible_matches_ids, left_on=['Unnamed: 0'], right_on=['Unnamed: 0'])
possible_matches = possible_matches.loc[((possible_matches.src_id==nodeid) & (possible_matches.tgt_id.isin(get_possible_matches(nodeid)))) | ((possible_matches.tgt_id==nodeid) & (possible_matches.src_id.isin(get_possible_matches(nodeid))))]
possible_matches

,Unnamed: 0,src_0,src_1,src_2,src_3,src_4,src_5,src_6,src_7,src_8,...,syntactic_diff,plus_diff,src_tgt_angle,src_angle_to_origin,tgt_angle_to_origin,src_veclen,tgt_veclen,src_tgt_veclen,src_id,tgt_id
1093,1093,-0.73279,-0.202696,0.596104,0.952525,0.367132,0.493698,0.639448,0.806481,-0.03964,...,0.363636,0.0,0.641700,0.344651,-0.044992,2.932257,4.205573,3.233933,http://dbkwik.webdatacommons.org/darkscape/property/maxranged,http://dbkwik.webdatacommons.org/oldschoolrunescape/property/ranged
1342,1342,-0.73279,-0.202696,0.596104,0.952525,0.367132,0.493698,0.639448,0.806481,-0.03964,...,0.461538,0.0,0.561231,0.344651,0.198992,2.932257,1.569314,2.428111,http://dbkwik.webdatacommons.org/darkscape/property/maxranged,http://dbkwik.webdatacommons.org/oldschoolrunescape/property/absorbranged


In [692]:
import re
src_pattern = "src_\d+"
tgt_pattern = "tgt_\d+"
a = np.array(possible_matches[["src_" + str(i) for i in range(20)]].values.tolist())
b = np.array(possible_matches[["tgt_" + str(i) for i in range(20)]].values.tolist())
#possible_matches['src_tgt_angle'] = cosine_similarity(a,b)[0]
#possible_matchesx = possible_matches.loc[((possible_matches.src_id==nodeid) & (possible_matches.tgt_id.isin(get_possible_matches(nodeid)))) | ((possible_matches.tgt_id==nodeid) & (possible_matches.src_id.isin(get_possible_matches(nodeid))))]
#possible_matches['src_tgt_angle'] = cosine_similarity(a,b)[0]
#model.wv[get_possible_matches(nodeid)]
#src_origin = np.full((len(possible_matches), 20), 0.0000001)
#tgt_origin = np.full((len(possible_matches), 20), 0.0000001)
#euclidean_distances(a,b)[0]
import sklearn
#possible_matches['src_tgt_angle'] = euclidean_distances(a,b).diagonal()
possible_matches=possible_matches.loc[((possible_matches.src_id==nodeid) & (possible_matches.tgt_id.isin(get_possible_matches(nodeid)))) | ((possible_matches.tgt_id==nodeid) & (possible_matches.src_id.isin(get_possible_matches(nodeid))))]
possible_matches

,Unnamed: 0,src_0,src_1,src_2,src_3,src_4,src_5,src_6,src_7,src_8,...,syntactic_diff,plus_diff,src_tgt_angle,src_angle_to_origin,tgt_angle_to_origin,src_veclen,tgt_veclen,src_tgt_veclen,src_id,tgt_id
1093,1093,-0.73279,-0.202696,0.596104,0.952525,0.367132,0.493698,0.639448,0.806481,-0.03964,...,0.363636,0.0,0.641700,0.344651,-0.044992,2.932257,4.205573,3.233933,http://dbkwik.webdatacommons.org/darkscape/property/maxranged,http://dbkwik.webdatacommons.org/oldschoolrunescape/property/ranged
1342,1342,-0.73279,-0.202696,0.596104,0.952525,0.367132,0.493698,0.639448,0.806481,-0.03964,...,0.461538,0.0,0.561231,0.344651,0.198992,2.932257,1.569314,2.428111,http://dbkwik.webdatacommons.org/darkscape/property/maxranged,http://dbkwik.webdatacommons.org/oldschoolrunescape/property/absorbranged


In [693]:
print('Closest in terms of cosine similarity:')
#vecs = model.docvecs.doctag_syn0[np.array(get_possible_matches(nodeid))]
vecs = model.wv[get_possible_matches(nodeid)]
x = cosine_similarity(np.array([model.wv[nodeid]]), vecs)
x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
sorted_x = possible_matches.sort_values(by=['src_tgt_angle'], ascending=False)
#sorted_x = pd.DataFrame(x).T.sort_values(by=['src_tgt_angle'], ascending=False)
sorted_x.loc[:,'cos_score'] = 0
ctr = 1
#sorted_x.columns = ['cos_sim' if col==0 else col for col in sorted_x.columns]
#sorted_x['cos_sim'] = sorted_x['cos_sim'].astype('float64')
maximum = sorted_x.head(1).src_tgt_angle.values[0]
sorted_x.loc[:,'diff_to_max'] = 1.0 - sorted_x.loc[:, 'src_tgt_angle'] / maximum
for index, row in sorted_x.iterrows():
    print(row.tgt_id + " - " + str(row['src_tgt_angle']))
    sorted_x.loc[index, 'cos_score'] = row['cos_score'] + 1/ctr
    ctr += 1

Closest in terms of cosine similarity:
http://dbkwik.webdatacommons.org/oldschoolrunescape/property/ranged - 0.641699779943525
http://dbkwik.webdatacommons.org/oldschoolrunescape/property/absorbranged - 0.5612306568129051


In [682]:
print('Closest in terms of Euclidean distance:')
sorted_x2 = sorted_x
#vecs = model.wv[get_possible_matches(nodeid)]
#x = euclidean_distances(np.array([model.wv[nodeid]]), vecs)
#x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
#sorted_x = pd.DataFrame(x).T.sort_values(by=[0], ascending=True)
sorted_x = possible_matches.sort_values(by=['src_tgt_veclen'], ascending=True)
sorted_x.loc[:,'euclid_score'] = 0
ctr = 1
#sorted_x.columns = ['src_tgt_veclen' if col==0 else col for col in sorted_x.columns]
for index, row in sorted_x.iterrows():
    print(row.tgt_id + " - " + str(row['src_tgt_veclen']))
    sorted_x.loc[index, 'euclid_score'] = row['euclid_score'] + 1/ctr
    ctr += 1

Closest in terms of Euclidean distance:
http://dbkwik.webdatacommons.org/oldschoolrunescape/property/disadvantages - 3.923097120542292


In [683]:
import editdistance
print('Closest in terms of syntax:')
sorted_x3 = sorted_x
#vecs = model.wv[get_possible_matches(nodeid)]
def edits(v1, v2s):
    res = list()
    v1 = v1.split("/")[-1]
    for v2 in v2s:
        v2 = v2.split("/")[-1]
        res.append(editdistance.eval(v1, v2)/min(len(v1), len(v2)))
    return np.array([res])
#x = edits(nodeid, get_possible_matches(nodeid))
#x = np.concatenate((x, np.array([get_possible_matches(nodeid)])), axis=0)
#sorted_x = pd.DataFrame(x).T.sort_values(by=[0], ascending=True)
sorted_x = possible_matches.sort_values(by=['syntactic_diff'], ascending=True)
sorted_x.loc[:,'syntax_score'] = 0
ctr = 1
sorted_x.columns = ['syntactic_diff' if col==0 else col for col in sorted_x.columns]
for index, row in sorted_x.iterrows():
    print(row.tgt_id + " - " + str(row['syntactic_diff']))
    sorted_x.loc[index, 'syntax_score'] = row['syntax_score'] + 1/ctr
    ctr += 1

Closest in terms of syntax:
http://dbkwik.webdatacommons.org/oldschoolrunescape/property/disadvantages - 0.2142857142857143


In [438]:
print('Closest in sum:')
x = sorted_x3['euclid_score'].to_frame().merge(sorted_x2['cos_score'].to_frame().merge(sorted_x, left_index=True, right_index=True), left_index=True, right_index=True)
x.loc[:,'total_score'] = x['cos_score'] + x['syntax_score']  + x['euclid_score'] 
sorted_x = x.sort_values(by=['total_score'], ascending=False)
#sorted_x.columns = ['tgt_id' if col==1 else col for col in sorted_x.columns]
for index, row in sorted_x.loc[sorted_x.total_score == max(sorted_x.total_score.values),:].iterrows():
    matching_pair = pd.DataFrame([sorted_x.loc[index]])
    matching_pair.loc[:,'src_id'] = nodeid
    if matchings is None:
        matchings = matching_pair
    else:
        matchings = matchings.append(matching_pair, ignore_index=True)
    print(nodeid + "\t" + row.tgt_id + "\t" + str(row['total_score']) + "\t" + str(row['cos_score']) + "\t" + str(row['euclid_score']))

Closest in sum:
http://dbkwik.webdatacommons.org/darkscape/property/disadvantages	http://dbkwik.webdatacommons.org/oldschoolrunescape/property/disadvantages	2.5	1.0	0.5


In [439]:
sorted_x

,euclid_score,cos_score,Unnamed: 0,src_0,src_1,src_2,src_3,src_4,src_5,src_6,...,src_tgt_angle,src_angle_to_origin,tgt_angle_to_origin,src_veclen,tgt_veclen,src_tgt_veclen,src_id,tgt_id,syntax_score,total_score
642,0.5,1.0,642,-2.198334,1.663798,0.232235,2.082636,0.769459,1.073691,-1.183635,...,0.539429,0.160991,-0.237124,5.091975,6.210288,5.314588,http://dbkwik.webdatacommons.org/darkscape/property/disadvantages,http://dbkwik.webdatacommons.org/oldschoolrunescape/property/disadvantages,1.0,2.5
128,1.0,0.5,128,-2.198334,1.663798,0.232235,2.082636,0.769459,1.073691,-1.183635,...,0.321648,0.160991,-0.022438,5.091975,3.787678,4.727607,http://dbkwik.webdatacommons.org/darkscape/property/disadvantages,http://dbkwik.webdatacommons.org/oldschoolrunescape/property/advantages,0.5,2.0


In [335]:
matchings = matchings.sort_values(by=['syntax_diff'], ascending=True)
matchings.head()

,euclid_sim,tgt_id,euclid_score,cos_sim,cos_score,diff_to_max,syntax_diff,syntax_score,total_score,src_id
2,2.8200011,http://dbkwik.webdatacommons.org/darkscape/resource/void_knight_equipment,0.5,0.850656,0.5,0.066107,0.0,1.0,3.0,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/void_knight_equipment
0,2.2271655,http://dbkwik.webdatacommons.org/darkscape/resource/elite_void_knight_equipment,1.0,0.910871,1.0,0.000000,0.2857142857142857,0.5,2.5,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/void_knight_equipment
1,2.2271655,http://dbkwik.webdatacommons.org/darkscape/resource/elite_void_knight_equipment,1.0,0.910871,1.0,0.000000,0.2857142857142857,0.5,3.0,http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/void_knight_equipment


In [343]:
matchings = matchings.sort_values(by=['total_score', 'syntax_diff'], ascending=[False, True])
matchings.head()
ctr = 0
with open('C:/Users/D072202/DeepAnyMatch/DeepAnyMatch/result_data/current_process/matchings.csv', mode="w+") as f:
    f.write("src_id,tgt_id,prediction\n")
    while len(matchings) > 0:
        ctr += 1
        row = matchings.head(1)
        #print(row.src_id.values[0] + "," + row.tgt_id.values[0])
        f.write(row.src_id.values[0] + "," + row.tgt_id.values[0]+",1\n")
        matchings = matchings.loc[~(matchings.src_id == row.src_id.values[0]) & ~(matchings.tgt_id == row.tgt_id.values[0])]
        print(pd.DataFrame(row))

  euclid_sim  \
0  2.385854    

                                                                tgt_id  \
0  http://dbkwik.webdatacommons.org/oldschoolrunescape/resource/prayer   

   euclid_score   cos_sim  cos_score  diff_to_max syntax_diff  syntax_score  \
0  1.0           0.884973  1.0        0.0          0.0         1.0            

   total_score                                                      src_id  
0  4.0          http://dbkwik.webdatacommons.org/darkscape/resource/prayer  
